<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

In [1]:
# Import all necessary libraries.
import os
import cv2
import numpy as np
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = "../logs/"
MASK_RCNN_MODEL_PATH = "../models/mask_rcnn_numberplate_0700.h5"
OPTIONS_MODEL_PATH =  "../models/numberplate_options_2019_2_15.h5"

# If you use gpu version tensorflow please change model to gpu version named like *-gpu.h5
OCR_NP_UKR_TEXT =  "../models/anpr_ocr_ua_1_2_11-cpu.h5"
OCR_NP_EU_TEXT =  "../models/anpr_ocr_eu_2-cpu.h5"

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector, OptionsDetector, Detector, textPostprocessing, textPostprocessingAsync
from NomeroffNet.Base import convert_keras_to_freeze_pb

# Initialize npdetector with default configuration file.
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load(OPTIONS_MODEL_PATH)

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2014_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": OCR_NP_UKR_TEXT
    },
    "eu": {
        "for_regions": ["eu", "eu_ua_1995"],
        "model_path": OCR_NP_EU_TEXT
    }
})

Using TensorFlow backend.


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


### Convert keras options .h5 model to .pb def graph and optimize

In [3]:
import tensorflow as tf
import keras
from tensorflow.core.framework import graph_pb2
import numpy as np
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.training import saver as saver_lib
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from keras import backend as K
from tensorflow.python.tools.optimize_for_inference_lib import optimize_for_inference
from tensorflow.python.framework import dtypes

class GraphDefParser():
    def __init__(self, graph):
        self.graph = graph
        self.nodes = self.graph.node
        self.nodes_dict = {}
        for i, node in enumerate(self.nodes):
            self.nodes_dict[node.name] = i

    def if_delete(self, current_node, nodes_names_for_deleting):
        delete = False
        for nodes_name_for_del in nodes_names_for_deleting:
            if -1 != current_node.find(nodes_name_for_del):
                delete = True
        return delete

    def delete_nodes(self, outputs, nodes_names_for_deleting):
        self._delete_nodes(outputs, nodes_names_for_deleting)
        i = 0
        while i < len(self.nodes):
            current_node = self.nodes[i].name
            if self.if_delete(current_node, nodes_names_for_deleting):
                del self.nodes[i]
            else:
                i += 1
        return self.nodes

    def _delete_nodes(self, outputs, nodes_names_for_deleting, perent=None, last_norm = None):
        for output in outputs:
            current_node = self.nodes[self.nodes_dict[output]].name
            current_node_s = self.nodes[self.nodes_dict[output]]
            for i, inp in enumerate(current_node_s.input):
                if inp not in self.nodes_dict.keys():
                    del current_node_s.input[i]
            if last_norm != None:
                last = self.nodes[self.nodes_dict[last_norm]]
                for i, inp in enumerate(last.input):
                    if inp == perent:
                        last.input[i] = current_node
                        #print("replace ", perent, " on ", current_node)
            if self.if_delete(current_node, nodes_names_for_deleting):
                last_norm = last_norm
                self._delete_nodes(current_node_s.input, nodes_names_for_deleting, current_node, last_norm)
            else:
                last_norm = current_node
                self._delete_nodes(current_node_s.input, nodes_names_for_deleting, current_node, last_norm)

def display_nodes(nodes):
    for i, node in enumerate(nodes):
        print('%d %s %s' % (i, node.name, node.op))
        [print(u'└─── %d ─ %s' % (i, n)) for i, n in enumerate(node.input)]

def delete_training_layers_from_keras(model):
    i = 0
    while i < len(model.layers):
        k = model.layers[i]
        if type(k) is keras.layers.Dropout:
            model.layers.remove(k)
        elif type(k) is keras.layers.BatchNormalization:
            model.layers.remove(k)
        elif  type(k) is keras.engine.training.Model:
            delete_training_layers_from_keras(k)
            i += 1
        else:
            #print(k.name, ":::",[inp.name for inp in k.input])
            i += 1
    return model

def convert_keras_to_freeze_pb(model, frozen_model_path, optimize = False, delete = []):
    model.trainable = False
    #model = delete_training_layers_from_keras(model)
    INPUT_NODE = [layer.name.split(":")[0]  for layer in model.outputs]
    OUTPUT_NODES = [layer.name.split(":")[0]  for layer in model.inputs]
    out_names = ",".join(INPUT_NODE)
    inp_names = ",".join(OUTPUT_NODES)
    print("OUTPUT: {}".format(out_names))
    print("INPUT: {}".format(inp_names))
    model.summary()
    K.set_learning_phase(0)
    sess = K.get_session()
    saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)
    checkpoint_path = saver.save(sess, './saved_ckpt', global_step=0, latest_filename='checkpoint_state')

    graph_io.write_graph(sess.graph, '.', './tmp.pb')
    freeze_graph.freeze_graph('./tmp.pb', '',
                            False, checkpoint_path, out_names,
                            "save/restore_all",
                            "save/Const:0",
                            frozen_model_path,
                            False, "")
    # load
    graph_def = tf.GraphDef()
    with tf.gfile.GFile(frozen_model_path, "rb") as f:
        graph_def.ParseFromString(f.read())
    print("FROZEN NODES")
    display_nodes(graph_def.node)

    optimized_graph_def = graph_def

    if bool(delete):
        # delete layers
        graphDefParser = GraphDefParser(optimized_graph_def)
        nodes = graphDefParser.delete_nodes(OUTPUT_NODES, delete)
        print("NODES AFTER DELETING")
        display_nodes(nodes)
        optimized_graph_def = graph_pb2.GraphDef()
        optimized_graph_def.node.extend(nodes)

    if optimize:
        # optimize
        optimized_graph_def = optimize_for_inference(graph_def,
                                   INPUT_NODE,
                                   OUTPUT_NODES,
                                   dtypes.float32.as_datatype_enum)
        print("OPTIMIZED NODES")
        display_nodes(optimized_graph_def.node)

    with tf.gfile.GFile(frozen_model_path, 'w') as f:
        f.write(optimized_graph_def.SerializeToString())

In [4]:
OPTION_DETECTOR_FROZEN_MODEL_PATH = "../models/numberplate_options_2019_2_15.pb"

convert_keras_to_freeze_pb(optionsDetector.MODEL, OPTION_DETECTOR_FROZEN_MODEL_PATH)

OUTPUT: REGION/Softmax,STATE/Softmax
INPUT: input_2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 295, 3)   0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    input_2[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 9216)         0           vgg16[1][0]                      
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 9216)         0           vgg16[1][0]                      
_________________________________________________________

In [5]:
graph_def = tf.GraphDef()
with tf.gfile.GFile(OPTION_DETECTOR_FROZEN_MODEL_PATH, "rb") as f:
    graph_def.ParseFromString(f.read())
optimized_graph_def = graph_def


# delete layers
graphDefParser = GraphDefParser(optimized_graph_def)
nodes = graphDefParser.delete_nodes(["REGION/Softmax", "STATE/Softmax"], ["dropout", "batch_normalization"])
print("NODES AFTER DELETING")
display_nodes(nodes)
optimized_graph_def = graph_pb2.GraphDef()
optimized_graph_def.node.extend(nodes)

NODES AFTER DELETING
0 input_2 Placeholder
1 block1_conv1/kernel Const
2 block1_conv1/kernel/read Identity
└─── 0 ─ block1_conv1/kernel
3 block1_conv1/bias Const
4 block1_conv1/bias/read Identity
└─── 0 ─ block1_conv1/bias
5 block1_conv2/kernel Const
6 block1_conv2/kernel/read Identity
└─── 0 ─ block1_conv2/kernel
7 block1_conv2/bias Const
8 block1_conv2/bias/read Identity
└─── 0 ─ block1_conv2/bias
9 block2_conv1/kernel Const
10 block2_conv1/kernel/read Identity
└─── 0 ─ block2_conv1/kernel
11 block2_conv1/bias Const
12 block2_conv1/bias/read Identity
└─── 0 ─ block2_conv1/bias
13 block2_conv2/kernel Const
14 block2_conv2/kernel/read Identity
└─── 0 ─ block2_conv2/kernel
15 block2_conv2/bias Const
16 block2_conv2/bias/read Identity
└─── 0 ─ block2_conv2/bias
17 block3_conv1/kernel Const
18 block3_conv1/kernel/read Identity
└─── 0 ─ block3_conv1/kernel
19 block3_conv1/bias Const
20 block3_conv1/bias/read Identity
└─── 0 ─ block3_conv1/bias
21 block3_conv2/kernel Const
22 block3_conv2/k

In [6]:
with tf.gfile.GFile("../models/numberplate_options_2019_2_15_2.pb", 'w') as f:
    f.write(optimized_graph_def.SerializeToString())

In [7]:
 with tf.gfile.GFile("../models/numberplate_options_2019_2_15_2.pb", "rb") as f:
    graph_def.ParseFromString(f.read())
print("FROZEN NODES")
display_nodes(graph_def.node)

FROZEN NODES
0 input_2 Placeholder
1 block1_conv1/kernel Const
2 block1_conv1/kernel/read Identity
└─── 0 ─ block1_conv1/kernel
3 block1_conv1/bias Const
4 block1_conv1/bias/read Identity
└─── 0 ─ block1_conv1/bias
5 block1_conv2/kernel Const
6 block1_conv2/kernel/read Identity
└─── 0 ─ block1_conv2/kernel
7 block1_conv2/bias Const
8 block1_conv2/bias/read Identity
└─── 0 ─ block1_conv2/bias
9 block2_conv1/kernel Const
10 block2_conv1/kernel/read Identity
└─── 0 ─ block2_conv1/kernel
11 block2_conv1/bias Const
12 block2_conv1/bias/read Identity
└─── 0 ─ block2_conv1/bias
13 block2_conv2/kernel Const
14 block2_conv2/kernel/read Identity
└─── 0 ─ block2_conv2/kernel
15 block2_conv2/bias Const
16 block2_conv2/bias/read Identity
└─── 0 ─ block2_conv2/bias
17 block3_conv1/kernel Const
18 block3_conv1/kernel/read Identity
└─── 0 ─ block3_conv1/kernel
19 block3_conv1/bias Const
20 block3_conv1/bias/read Identity
└─── 0 ─ block3_conv1/bias
21 block3_conv2/kernel Const
22 block3_conv2/kernel/re

### Convert keras OCR .h5 model to .pb def graph and optimize

In [8]:
from NomeroffNet.Base import OCR

OCR_NP_UKR_FROZEN_MODEL_PATH = "../models/anpr_ocr_ua_1_2_11-cpu.pb"
model = textDetector.get_module("eu_ua_2014_2015").MODEL
convert_keras_to_freeze_pb(model, OCR_NP_UKR_FROZEN_MODEL_PATH)

OUTPUT: softmax/truediv
INPUT: the_input
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
____________________________________________________________________

In [9]:
OCR_NP_EU_FROZEN_MODEL_PATH = "../models/anpr_ocr_eu_2-cpu.pb"
model = textDetector.get_module("eu").MODEL
convert_keras_to_freeze_pb(model, OCR_NP_EU_FROZEN_MODEL_PATH)

OUTPUT: softmax_1/truediv
INPUT: the_input_1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
________________________________________________________________

### Convert MaskRCNN .h5 model  to frozen .pb def graph and optimize

In [10]:
MASK_RCNN_FROZEN_MODEL_PATH = "../models/mask_rcnn_numberplate_0700.pb"
model = nnet.getKerasModel()

convert_keras_to_freeze_pb(model,MASK_RCNN_FROZEN_MODEL_PATH)

OUTPUT: mrcnn_detection/Reshape_1,mrcnn_class/Reshape_1,mrcnn_bbox/Reshape,mrcnn_mask/Reshape_1,ROI/packed_2,rpn_class/concat,rpn_bbox/concat
INPUT: input_image,input_image_meta,input_anchors
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchN

INFO:tensorflow:Restoring parameters from ./saved_ckpt-0
INFO:tensorflow:Froze 690 variables.
INFO:tensorflow:Converted 690 variables to const ops.
FROZEN NODES
0 input_image Placeholder
1 input_image_meta Placeholder
2 input_anchors Placeholder
3 zero_padding2d_1/Pad/paddings Const
4 zero_padding2d_1/Pad Pad
└─── 0 ─ input_image
└─── 1 ─ zero_padding2d_1/Pad/paddings
5 conv1/kernel Const
6 conv1/kernel/read Identity
└─── 0 ─ conv1/kernel
7 conv1/bias Const
8 conv1/bias/read Identity
└─── 0 ─ conv1/bias
9 conv1/convolution Conv2D
└─── 0 ─ zero_padding2d_1/Pad
└─── 1 ─ conv1/kernel/read
10 conv1/BiasAdd BiasAdd
└─── 0 ─ conv1/convolution
└─── 1 ─ conv1/bias/read
11 bn_conv1/gamma Const
12 bn_conv1/gamma/read Identity
└─── 0 ─ bn_conv1/gamma
13 bn_conv1/beta Const
14 bn_conv1/beta/read Identity
└─── 0 ─ bn_conv1/beta
15 bn_conv1/moving_mean Const
16 bn_conv1/moving_mean/read Identity
└─── 0 ─ bn_conv1/moving_mean
17 bn_conv1/moving_variance Const
18 bn_conv1/moving_variance/read Identity

1509 res4w_branch2b/BiasAdd BiasAdd
└─── 0 ─ res4w_branch2b/convolution
└─── 1 ─ res4w_branch2b/bias/read
1510 bn4w_branch2b/gamma Const
1511 bn4w_branch2b/gamma/read Identity
└─── 0 ─ bn4w_branch2b/gamma
1512 bn4w_branch2b/beta Const
1513 bn4w_branch2b/beta/read Identity
└─── 0 ─ bn4w_branch2b/beta
1514 bn4w_branch2b/moving_mean Const
1515 bn4w_branch2b/moving_mean/read Identity
└─── 0 ─ bn4w_branch2b/moving_mean
1516 bn4w_branch2b/moving_variance Const
1517 bn4w_branch2b/moving_variance/read Identity
└─── 0 ─ bn4w_branch2b/moving_variance
1518 bn4w_branch2b/FusedBatchNorm FusedBatchNorm
└─── 0 ─ res4w_branch2b/BiasAdd
└─── 1 ─ bn4w_branch2b/gamma/read
└─── 2 ─ bn4w_branch2b/beta/read
└─── 3 ─ bn4w_branch2b/moving_mean/read
└─── 4 ─ bn4w_branch2b/moving_variance/read
1519 activation_61/Relu Relu
└─── 0 ─ bn4w_branch2b/FusedBatchNorm
1520 res4w_branch2c/kernel Const
1521 res4w_branch2c/kernel/read Identity
└─── 0 ─ res4w_branch2c/kernel
1522 res4w_branch2c/bias Const
1523 res4w_branch2

└─── 1 ─ mrcnn_mask_conv4/kernel/read
2885 mrcnn_mask_conv4/BiasAdd BiasAdd
└─── 0 ─ mrcnn_mask_conv4/convolution
└─── 1 ─ mrcnn_mask_conv4/bias/read
2886 mrcnn_mask_conv4/Reshape_1/shape Const
2887 mrcnn_mask_conv4/Reshape_1 Reshape
└─── 0 ─ mrcnn_mask_conv4/BiasAdd
└─── 1 ─ mrcnn_mask_conv4/Reshape_1/shape
2888 mrcnn_mask_bn4/gamma Const
2889 mrcnn_mask_bn4/gamma/read Identity
└─── 0 ─ mrcnn_mask_bn4/gamma
2890 mrcnn_mask_bn4/beta Const
2891 mrcnn_mask_bn4/beta/read Identity
└─── 0 ─ mrcnn_mask_bn4/beta
2892 mrcnn_mask_bn4/moving_mean Const
2893 mrcnn_mask_bn4/moving_mean/read Identity
└─── 0 ─ mrcnn_mask_bn4/moving_mean
2894 mrcnn_mask_bn4/moving_variance Const
2895 mrcnn_mask_bn4/moving_variance/read Identity
└─── 0 ─ mrcnn_mask_bn4/moving_variance
2896 mrcnn_mask_bn4/Reshape/shape Const
2897 mrcnn_mask_bn4/Reshape Reshape
└─── 0 ─ mrcnn_mask_conv4/Reshape_1
└─── 1 ─ mrcnn_mask_bn4/Reshape/shape
2898 mrcnn_mask_bn4/FusedBatchNorm FusedBatchNorm
└─── 0 ─ mrcnn_mask_bn4/Reshape
└─── 